<a href="https://colab.research.google.com/github/Suman-programmer/DS_Hackathon/blob/main/NLP_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import numpy as np
import pandas as pd

In [41]:
import tensorflow as tf

In [42]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils.vis_utils import plot_model

In [43]:
import nltk
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [47]:
data = pd.read_excel("Hackathon_Training_data.xlsx")

In [48]:
data.head()

,id,code,docstring,y_true
0,66236bf5-ce26-4401-82e6-1660c718008f,"def get_uids(self, filename=None):\n se...",Return a list of UIDs\n filename -- un...,1
1,b6a4ef2c-8109-486c-a9ad-4f20d2c95fb2,"def purge_cdn_object(self, container, obj, ema...",Removes a CDN-enabled object from public acces...,1
2,18a1cd0f-79e4-4977-a949-64695ffdf8d0,"def print_tree(self, tree=None):\n if t...",Update running digest with content of named file.,0
3,db2f8458-8e19-4037-b8c4-4b2c9ce468ef,"def sign_bitcoin(self, message, compressed=Fal...",Returns a go type according to a spec type,0
4,f0a3aa4e-35b1-4b7a-a194-74ae6967ceee,"def index(self, val, start=None, stop=None):\n...",Return the smallest *k* such that L[k] == val ...,1


In [49]:
data.shape

(100000, 4)

In [50]:
#Text preprocesing, removes punctuations from code and docstring
def remove_punc(text_data) :
    text_data = text_data.translate(str.maketrans('', '', string.punctuation))
    return text_data

In [51]:
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop.extend(['I', 'is',])
import string
!pip install contractions
import contractions

# expand contractions
data['docstring']=data['docstring'].astype(str)
data['cleaned_docstring'] = data['docstring'].str.lower().apply(contractions.fix)

# remove stop words
data['cleaned_docstring'] = data['cleaned_docstring'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))

# remove numbers + escape characters
data['cleaned_docstring'] = data['cleaned_docstring'].apply(lambda x: ' '.join([word for word in x.split() if word.isalpha()]))

# remove punctuation
data['cleaned_docstring'] = data['cleaned_docstring'].apply(lambda x: ''.join([item for item in x if item not in string.punctuation]))

# tokenize after cleaning
data['tokenized_docstring'] = data['cleaned_docstring'].apply(nltk.word_tokenize)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
data.head()

,id,code,docstring,y_true,cleaned_docstring,tokenized_docstring
0,66236bf5-ce26-4401-82e6-1660c718008f,"def get_uids(self, filename=None):\n se...",Return a list of UIDs\n filename -- un...,1,return list uids filename api compatibility,"[return, list, uids, filename, api, compatibil..."
1,b6a4ef2c-8109-486c-a9ad-4f20d2c95fb2,"def purge_cdn_object(self, container, obj, ema...",Removes a CDN-enabled object from public acces...,1,removes object public access ttl please note l...,"[removes, object, public, access, ttl, please,..."
2,18a1cd0f-79e4-4977-a949-64695ffdf8d0,"def print_tree(self, tree=None):\n if t...",Update running digest with content of named file.,0,update running digest content named,"[update, running, digest, content, named]"
3,db2f8458-8e19-4037-b8c4-4b2c9ce468ef,"def sign_bitcoin(self, message, compressed=Fal...",Returns a go type according to a spec type,0,returns go type according spec type,"[returns, go, type, according, spec, type]"
4,f0a3aa4e-35b1-4b7a-a194-74ae6967ceee,"def index(self, val, start=None, stop=None):\n...",Return the smallest *k* such that L[k] == val ...,1,return smallest val k raises valueerror defaul...,"[return, smallest, val, k, raises, valueerror,..."


In [53]:
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# stop = stopwords.words('english')
# stop.extend(['I', 'is',])
# import string
# !pip install contractions
# import contractions

# expand contractions
data['code']=data['code'].astype(str)
data['cleaned_code'] = data['code'].str.lower().apply(contractions.fix)

# remove stop words
data['cleaned_code'] = data['cleaned_code'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))

# remove numbers + escape characters
data['cleaned_code'] = data['cleaned_code'].apply(lambda x: ' '.join([word for word in x.split() if word.isalpha()]))

# remove punctuation
data['cleaned_code'] = data['cleaned_code'].apply(lambda x: ''.join([item for item in x if item not in string.punctuation]))

# tokenize after cleaning
data['tokenized_code'] = data['cleaned_code'].apply(nltk.word_tokenize)

In [54]:
data.head()

,id,code,docstring,y_true,cleaned_docstring,tokenized_docstring,cleaned_code,tokenized_code
0,66236bf5-ce26-4401-82e6-1660c718008f,"def get_uids(self, filename=None):\n se...",Return a list of UIDs\n filename -- un...,1,return list uids filename api compatibility,"[return, list, uids, filename, api, compatibil...",def return entry,"[def, return, entry]"
1,b6a4ef2c-8109-486c-a9ad-4f20d2c95fb2,"def purge_cdn_object(self, container, obj, ema...",Removes a CDN-enabled object from public acces...,1,removes object public access ttl please note l...,"[removes, object, public, access, ttl, please,...",def return,"[def, return]"
2,18a1cd0f-79e4-4977-a949-64695ffdf8d0,"def print_tree(self, tree=None):\n if t...",Update running digest with content of named file.,0,update running digest content named,"[update, running, digest, content, named]",def tree,"[def, tree]"
3,db2f8458-8e19-4037-b8c4-4b2c9ce468ef,"def sign_bitcoin(self, message, compressed=Fal...",Returns a go type according to a spec type,0,returns go type according spec type,"[returns, go, type, according, spec, type]",def return,"[def, return]"
4,f0a3aa4e-35b1-4b7a-a194-74ae6967ceee,"def index(self, val, start=None, stop=None):\n...",Return the smallest *k* such that L[k] == val ...,1,return smallest val k raises valueerror defaul...,"[return, smallest, val, k, raises, valueerror,...",def raise start start start start start start ...,"[def, raise, start, start, start, start, start..."


In [55]:
# stemming/lemmatization (change words to base words)

import nltk
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# function from link in comment
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence, ' '.join(lemmatized_sentence)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [56]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [57]:

data['lemmatized_docstring'] = data['tokenized_docstring'].apply(lemmatize_sentence)
data['lemmatized_code'] = data['tokenized_code'].apply(lemmatize_sentence)

In [58]:
data.head()

,id,code,docstring,y_true,cleaned_docstring,tokenized_docstring,cleaned_code,tokenized_code,lemmatized_docstring,lemmatized_code
0,66236bf5-ce26-4401-82e6-1660c718008f,"def get_uids(self, filename=None):\n se...",Return a list of UIDs\n filename -- un...,1,return list uids filename api compatibility,"[return, list, uids, filename, api, compatibil...",def return entry,"[def, return, entry]","([return, list, uids, filename, api, compatibi...","([def, return, entry], def return entry)"
1,b6a4ef2c-8109-486c-a9ad-4f20d2c95fb2,"def purge_cdn_object(self, container, obj, ema...",Removes a CDN-enabled object from public acces...,1,removes object public access ttl please note l...,"[removes, object, public, access, ttl, please,...",def return,"[def, return]","([remove, object, public, access, ttl, please,...","([def, return], def return)"
2,18a1cd0f-79e4-4977-a949-64695ffdf8d0,"def print_tree(self, tree=None):\n if t...",Update running digest with content of named file.,0,update running digest content named,"[update, running, digest, content, named]",def tree,"[def, tree]","([update, run, digest, content, name], update ...","([def, tree], def tree)"
3,db2f8458-8e19-4037-b8c4-4b2c9ce468ef,"def sign_bitcoin(self, message, compressed=Fal...",Returns a go type according to a spec type,0,returns go type according spec type,"[returns, go, type, according, spec, type]",def return,"[def, return]","([return, go, type, accord, spec, type], retur...","([def, return], def return)"
4,f0a3aa4e-35b1-4b7a-a194-74ae6967ceee,"def index(self, val, start=None, stop=None):\n...",Return the smallest *k* such that L[k] == val ...,1,return smallest val k raises valueerror defaul...,"[return, smallest, val, k, raises, valueerror,...",def raise start start start start start start ...,"[def, raise, start, start, start, start, start...","([return, small, val, k, raise, valueerror, de...","([def, raise, start, start, start, start, star..."


In [59]:
data['cleaned_docstring'] = data['cleaned_docstring'].apply(remove_punc)

In [60]:
data['cleaned_code'] = data['cleaned_code'].apply(remove_punc)

In [61]:
#Split data into train and val data
train, val = train_test_split(data, test_size = 0.3, random_state = 123)

In [62]:
#Set max vector length
docstring_tfidf_vec_len = 1000
code_tfidf_vec_len = 1000

In [63]:
#Initialise tfidf vectorizers and fit for code and docstring column values
tf_vec_docstring = TfidfVectorizer(max_features = docstring_tfidf_vec_len)
tf_vec_code = TfidfVectorizer(max_features = code_tfidf_vec_len)

In [64]:
tf_vec_docstring.fit(train['cleaned_docstring'].values)
tf_vec_code.fit(train['cleaned_code'].values)

TfidfVectorizer(max_features=1000)

In [65]:
#Save TFIDF Vectorizer objects IN THE SAME PATH AS THE SCORING AND SUBMISSION NOTEBOOKS
joblib.dump(tf_vec_docstring, r'.\tf_vec_docstring.pkl')
joblib.dump(tf_vec_code, r'.\tf_vec_code.pkl')

['.\\tf_vec_code.pkl']

In [66]:
#Load tfidf vectorizer objects
tf_vec_docstring = joblib.load(r'.\tf_vec_docstring.pkl')
tf_vec_code = joblib.load(r'.\tf_vec_code.pkl')

In [67]:
#Transform train data to vectors
x_train_code = tf_vec_code.transform(train['cleaned_code']).toarray()
x_train_docstring = tf_vec_docstring.transform(train['cleaned_docstring']).toarray()

In [68]:
#Transform val data to vectors
x_val_code = tf_vec_code.transform(val['cleaned_code']).toarray()
x_val_docstring = tf_vec_docstring.transform(val['cleaned_docstring']).toarray()

In [69]:
#Set labels as np array
y_train = train['y_true'].values
y_val = val['y_true'].values

In [70]:
#Delete train and val objects to optimize memory usage
del train
del val

In [72]:
#Create input for Keras models
input_docstring = tf.keras.Input(shape = (x_train_docstring.shape[1]))
input_code = tf.keras.Input(shape = (x_train_code.shape[1]))

In [73]:
x_docstring = layers.Dense(50, activation = 'tanh', kernel_initializer ='he_normal')(input_docstring)
x_docstring = layers.Dense(25, activation = 'relu', kernel_initializer ='he_normal')(x_docstring)

In [74]:
x_code = layers.Dense(50, activation = 'tanh', kernel_initializer ='he_normal')(input_code)
x_code = layers.Dense(25, activation = 'relu', kernel_initializer ='he_normal')(x_code)

In [75]:
#Create final ouput layer with sigmiod activation
x_concat = layers.Concatenate(axis = -1)([x_docstring, x_code])
output = layers.Dense(1, activation = 'sigmoid')(x_concat)

In [76]:
#Define Keras model
model = tf.keras.models.Model(inputs = ([input_docstring, input_code]), outputs = output)

In [77]:
#Model Summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1000)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 1000)]       0           []                               
                                                                                                  
 dense_5 (Dense)                (None, 50)           50050       ['input_3[0][0]']                
                                                                                                  
 dense_7 (Dense)                (None, 50)           50050       ['input_4[0][0]']                
                                                                                            

In [78]:
#Define model optimizer and create full model
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])

In [79]:
#Train model for 5 epochs
out = model.fit(x = [x_train_docstring, x_train_code], y = y_train, epochs = 300, batch_size = 256, validation_data = ([x_val_docstring, x_val_code], y_val))

Epoch 1/300
274/274 [==============================] - 3s 10ms/step - loss: 0.5689 - accuracy: 0.7503 - val_loss: 0.5587 - val_accuracy: 0.7551
Epoch 2/300
274/274 [==============================] - 2s 9ms/step - loss: 0.5578 - accuracy: 0.7507 - val_loss: 0.5625 - val_accuracy: 0.7551
Epoch 3/300
274/274 [==============================] - 2s 7ms/step - loss: 0.5546 - accuracy: 0.7507 - val_loss: 0.5629 - val_accuracy: 0.7551
Epoch 4/300
274/274 [==============================] - 2s 7ms/step - loss: 0.5517 - accuracy: 0.7507 - val_loss: 0.5641 - val_accuracy: 0.7551
Epoch 5/300
274/274 [==============================] - 2s 6ms/step - loss: 0.5483 - accuracy: 0.7508 - val_loss: 0.5662 - val_accuracy: 0.7547
Epoch 6/300
274/274 [==============================] - 2s 7ms/step - loss: 0.5432 - accuracy: 0.7514 - val_loss: 0.5689 - val_accuracy: 0.7537
Epoch 7/300
274/274 [==============================] - 2s 7ms/step - loss: 0.5358 - accuracy: 0.7528 - val_loss: 0.5730 - val_accuracy: 0.752

In [80]:
#Save model object
model.save(r'\model_1.h5')

In [81]:
pred_prob = model.predict([x_val_docstring, x_val_code])

938/938 [==============================] - 1s 1ms/step


In [82]:
pred_class = (pred_prob >=0.70)*1

In [83]:
#calculating F1
from sklearn.metrics import classification_report
print(classification_report(y_val,pred_class))

              precision    recall  f1-score   support

           0       0.25      0.27      0.26      7348
           1       0.76      0.74      0.75     22652

    accuracy                           0.62     30000
   macro avg       0.50      0.50      0.50     30000
weighted avg       0.63      0.62      0.63     30000

